In [95]:
import pandas as pd
from unidecode import unidecode

In [96]:
wiki = pd.read_csv("../datasets/wiki_movie_plots_deduped.csv")
wiki

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [97]:
train = pd.read_csv("../datasets/imdb/complete_train.csv", sep=',')
train

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True
...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,NaN,2019,\N,87,12951.0,False
7955,9981,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False
7956,9982,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,\N,111,4144.0,True


In [98]:
train["startYear"].replace(r"\N", None, inplace=True)
train["startYear"].fillna(train['endYear'], inplace=True)
train["startYear"] = train["startYear"].astype(int)

In [99]:
train["primaryTitle"] = train["primaryTitle"].str.lower()
train['primaryTitle'] = train['primaryTitle'].apply(unidecode)

In [100]:
wiki["Title"] = wiki["Title"].str.lower()
wiki.rename({"Title": "primaryTitle"}, axis=1, inplace=True)
wiki.rename({"Release Year": "startYear"}, axis=1, inplace=True)
wiki['primaryTitle'] = wiki['primaryTitle'].apply(unidecode)

In [101]:
wiki[wiki["primaryTitle"] == "the doll"]

,startYear,primaryTitle,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot


In [102]:
pd.merge(train, wiki[["primaryTitle", "Genre", "startYear", "Plot"]],on=["primaryTitle", "startYear"], how="left")

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,Genre,Plot
0,4,tt0010600,the doll,Die Puppe,1919,\N,66,1898.0,True,NaN,NaN
1,7,tt0011841,way down east,Way Down East,1920,\N,145,5376.0,True,drama,"The rich, typified by the handsome man-about-t..."
2,9,tt0012494,destiny,Der müde Tod,1921,\N,97,5842.0,True,NaN,NaN
3,25,tt0015163,the navigator,The Navigator,1924,\N,59,9652.0,True,comedy,Wealthy Rollo Treadway (Buster Keaton) suddenl...
4,38,tt0016220,the phantom of the opera,The Phantom of the Opera,1925,\N,93,17887.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7983,9966,tt9625664,trauma center,NaN,2019,\N,87,12951.0,False,NaN,NaN
7984,9981,tt9741310,slaxx,Slaxx,2020,\N,77,2464.0,False,NaN,NaN
7985,9982,tt9742392,kindred,Kindred,2020,\N,101,1719.0,False,NaN,NaN
7986,9996,tt9850386,the bee gees: how can you mend a broken heart,NaN,2020,\N,111,4144.0,True,NaN,NaN


### Validation

In [104]:
validation = pd.read_csv("../datasets/imdb/validation_hidden.csv", sep=',')
validation

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes
0,0,tt0003740,Cabiria,NaN,1914,\N,148,3452.0
1,1,tt0008663,A Man There Was,Terje Vigen,1917,\N,65,1882.0
2,3,tt0010307,J'accuse!,NaN,1919,\N,166,1692.0
3,18,tt0014429,Safety Last!,Safety Last!,1923,\N,74,19898.0
4,27,tt0015175,Die Nibelungen: Siegfried,NaN,1924,\N,143,5676.0
...,...,...,...,...,...,...,...,...
950,9974,tt9686154,You Will Die at 20,NaN,2019,\N,103,2106.0
951,9976,tt9690328,Pápér Spidérs,Paper Spiders,2020,\N,109,NaN
952,9980,tt9735790,Me You Madness,Me You Madness,2021,\N,98,1056.0
953,9984,tt9769668,Tughlaq Durbar,Tughlaq Durbar,2021,\N,145,1430.0


In [105]:
validation["startYear"].replace(r"\N", None, inplace=True)
validation["startYear"].fillna(validation['endYear'], inplace=True)
validation["startYear"] = validation["startYear"].astype(int)

In [106]:
validation["primaryTitle"] = validation["primaryTitle"].str.lower()
validation['primaryTitle'] = validation['primaryTitle'].apply(unidecode)

In [107]:
pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear", "Plot"]],on=["primaryTitle", "startYear"], how="left")

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,Genre,Plot
0,0,tt0003740,cabiria,NaN,1914,\N,148,3452.0,NaN,NaN
1,1,tt0008663,a man there was,Terje Vigen,1917,\N,65,1882.0,NaN,NaN
2,3,tt0010307,j'accuse!,NaN,1919,\N,166,1692.0,NaN,NaN
3,18,tt0014429,safety last!,Safety Last!,1923,\N,74,19898.0,comedy,The film opens in 1922 with Harold Lloyd (the ...
4,27,tt0015175,die nibelungen: siegfried,NaN,1924,\N,143,5676.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
954,9974,tt9686154,you will die at 20,NaN,2019,\N,103,2106.0,NaN,NaN
955,9976,tt9690328,paper spiders,Paper Spiders,2020,\N,109,NaN,NaN,NaN
956,9980,tt9735790,me you madness,Me You Madness,2021,\N,98,1056.0,NaN,NaN
957,9984,tt9769668,tughlaq durbar,Tughlaq Durbar,2021,\N,145,1430.0,NaN,NaN


### Test data